In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model = AutoModelForCausalLM.from_pretrained("/root/Logical-Fallacies/models/arguments/cpo_Llama-2-7b-hf_2024-06-02_15:14:57.412945/checkpoint-500", device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("/root/Logical-Fallacies/models/arguments/cpo_Llama-2-7b-hf_2024-06-02_15:14:57.412945/checkpoint-500")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [46]:
class_weights = {0: 0, 1: 1135, 2: 674, 3: 532, 4: 553, 5: 194, 6: 728, 7: 527, 8: 848, 9: 570, 10: 517, 11: 479, 12: 429, 13: 449}

for k, v in class_weights.items():
    if k != 0:
        class_weights[k] = round(v/(7636), 2)
    
print(class_weights)
print(sum(list(class_weights.values())))
class_weights.values()


{0: 0, 1: 0.15, 2: 0.09, 3: 0.07, 4: 0.07, 5: 0.03, 6: 0.1, 7: 0.07, 8: 0.11, 9: 0.07, 10: 0.07, 11: 0.06, 12: 0.06, 13: 0.06}
1.0100000000000002


dict_values([0, 0.15, 0.09, 0.07, 0.07, 0.03, 0.1, 0.07, 0.11, 0.07, 0.07, 0.06, 0.06, 0.06])

In [2]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=256, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=256, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): Mod

In [5]:
for param in model.named_parameters():
    print(param[0])

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.base_layer.weight
model.layers.0.self_attn.q_proj.lora_A.default.weight
model.layers.0.self_attn.q_proj.lora_B.default.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.base_layer.weight
model.layers.0.self_attn.v_proj.lora_A.default.weight
model.layers.0.self_attn.v_proj.lora_B.default.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.base_layer.weight
model.layers.1.self_attn.q_proj.lora_A.default.weight
model.layers.1.self_attn.q_proj.lora_B.default.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.base_layer.weight
model.layers.1.self_attn.v_proj.lora_A.default.weight
model.layers.1.self_attn.v_proj.lora_B.default.weight
model.layers.1.self_attn.o_proj.we

: 

In [3]:
def generate(prompt: str, model, tokenizer,n=5, **generate_kwargs):
        """Main function for each worker process (may be only 1 for BasicTrainer/TensorParallelTrainer)."""
        tokenized_prompt = tokenizer(prompt, return_tensors='pt', max_length=100, truncation=True).to(model.device)
        with torch.no_grad():
            output = model.generate(**tokenized_prompt,
                                    **generate_kwargs,
                                    pad_token_id=tokenizer.eos_token_id,
                                    num_return_sequences=n)
            logits = model(**tokenized_prompt).logits
            clf = model.classification_head(logits)
            clf = torch.softmax(clf, dim=1)
        output_decoded = tokenizer.batch_decode(output, skip_special_tokens=True)
        return output_decoded, clf

In [4]:

topic = "Cannabis should be legalized."
stance = 'SUPPORTING' 
prompt = f"<s> [INST] ### Prompt:  Generate a {stance} argument for the topic: {topic} [/INST]\n### Argument: "

y = generate(prompt, model, tokenizer, n=1)[0]
print(y)

AttributeError: 'LlamaForCausalLM' object has no attribute 'classification_head'

In [ ]:
model.classification_head()